from langchain.llms import GooglePalm


In [6]:
from langchain.llms import GooglePalm
api_key = '
llm = GooglePalm(google_api_key=api_key, temperature=0.5)

In [7]:
poem = llm("Write a 4 line poem of my love for samosa")
print(poem)

**Oh, samosa, my love**
**You are so delicious,**
**With your crispy shell**
**And your flavorful filling.**


In [ ]:
essay = llm("write email requesting refund for electronic item")
print(essay)

Dear [Name of Customer Service Representative],

I am writing to request a refund for the [product name] that I purchased on [date]. I am not satisfied with the product because [explain why you are not satisfied].

I have attached a copy of my receipt and a picture of the product. I have also included a link to the product page on your website.

I would like to request a full refund for the product. I would also like to request that you send me a replacement product.

I would appreciate it if you could process my refund as soon as possible. I can be reached at [email address] or [phone number] if you have any questions.

Thank you for your time and consideration.

Sincerely,
[Your Name]


In [ ]:
essay = llm("write essay on Sri Lanka")
print(essay)

Sri Lanka, officially the Democratic Socialist Republic of Sri Lanka, is an island country in South Asia. It lies in the Indian Ocean, southwest of the Bay of Bengal and southeast of the Arabian Sea. Sri Lanka has a rich history, culture, and natural beauty. It is home to many ancient ruins, including the ruins of the ancient city of Anuradhapura, which was once the capital of the Sinhalese Kingdom. Sri Lanka is also home to many beautiful beaches, mountains, and rainforests.

The people of Sri Lanka are very diverse. The majority of the population is Sinhalese, but there are also significant Tamil, Muslim, and Burgher minorities. The official languages of Sri Lanka are Sinhala and Tamil. English is also widely spoken.

Sri Lanka has a long history of conflict between the Sinhalese and Tamil people. The conflict began in the 1980s and has resulted in the deaths of thousands of people. In 2009, the Sri Lankan government declared victory over the Tamil Tigers, the rebel group that had be

In [8]:
from langchain.document_loaders.csv_loader import CSVLoader

# Specify the encoding if known (e.g., 'latin1', 'windows-1252')
loader = CSVLoader(file_path='codebasics_faqs.csv', source_column="prompt", encoding='latin1')

# Store the loaded data in the 'data' variable
data = loader.load()

In [10]:
# ! pip install InstructorEmbedding
# ! pip install torch
# ! pip install sentence-transformers




     ---------------------------------------- 0.0/43.8 kB ? eta -:--:--
     --------- ------------------------------ 10.2/43.8 kB ? eta -:--:--
     ----------------- -------------------- 20.5/43.8 kB 320.0 kB/s eta 0:00:01
     -------------------------- ----------- 30.7/43.8 kB 262.6 kB/s eta 0:00:01
     -------------------------------------- 43.8/43.8 kB 268.6 kB/s eta 0:00:00
  Using cached huggingface_hub-0.23.3-py3-none-any.whl.metadata (12 kB)
   ---------------------------------------- 0.0/227.1 kB ? eta -:--:--
   ------- ------------------------------- 41.0/227.1 kB 991.0 kB/s eta 0:00:01
   -------------- ------------------------ 81.9/227.1 kB 919.0 kB/s eta 0:00:01
   ---------------------------- ----------- 163.8/227.1 kB 1.2 MB/s eta 0:00:01
   ---------------------------------------  225.3/227.1 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 227.1/227.1 kB 1.2 MB/s eta 0:00:00
Using cached huggingface_hub-0.23.3-py3-none-any.whl (401 kB)
   -------

In [9]:
from langchain.embeddings import GooglePalmEmbeddings

# Initialize instructor embeddings using the Hugging Face model
google_palm_embeddings = GooglePalmEmbeddings(google_api_key=api_key)


In [10]:
e = google_palm_embeddings.embed_query("What is your refund policy?")

In [18]:
len(e)

768

In [19]:
print(e[:5])

[-0.004683516, 0.02611447, 0.01816308, 0.003736937, 0.016299238]


In [11]:

from langchain.vectorstores import FAISS

# Create a FAISS instance for vector database from 'data'
vectordb = FAISS.from_documents(documents=data,
                                 embedding=google_palm_embeddings)

In [12]:
# Create a retriever for querying the vector database
retriever = vectordb.as_retriever(score_threshold = 0.7)

In [13]:
rdocs = retriever.get_relevant_documents("how about job placement support?")
rdocs

c:\Users\Administrator\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.', metadata={'source': 'Do you provide any job assistance?', 'row': 11}),
 Document(page_content='prompt: Do you provide any virtual internship?\nresponse: Yes', metadata={'source': 'Do you provide any virtual internship?', 'row': 14}),
 Document(page_content='prompt: Will this bootcamp guarantee me a job?\nresponse: The courses included in this bootcamp are done by 9000+ learners and many of them have secured a job which gives us ample confidence that you will be able to get a job. However, we want to be honest and do not want to make any impractical promises! Our guarantee is to prepare you for the job market by teaching the most relevant skills, knowledge & timeless principles good enough to fetch the job.', metadata={'sourc

In [14]:
from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}


from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=llm,
                            chain_type="stuff",
                            retriever=retriever,
                            input_key="query",
                            return_source_documents=True,
                            chain_type_kwargs=chain_type_kwargs)

In [15]:
chain('Do you provide job assistance and also do you provide job gurantee?')

c:\Users\Administrator\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'query': 'Do you provide job assistance and also do you provide job gurantee?',
 'result': 'Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.\nThe courses included in this bootcamp are done by 9000+ learners and many of them have secured a job which gives us ample confidence that you will be able to get a job. However, we want to be honest and do not want to make any impractical promises! Our guarantee is to prepare you for the job market by teaching the most relevant skills, knowledge & timeless principles good enough to fetch the job.',
 'source_documents': [Document(page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.', metadata={'source': 'Do you pro